In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mines-vs-rocks/sonar.all-data.csv


### Deep Learning Solution
* The aim behind this notebook is to implement and test the effect of Dropout on Neural Network.
* Dropout in Neural Network is used to overcome the problem of overfitting in NN
* It is a binary classification problem that requires a model to differentiate rocks from metal cylinders.

In [2]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')



2021-09-22 13:47:46.675230: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-09-22 13:47:46.675356: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Importing data
# As the data has no header, we set header to None
dataframe = pd.read_csv("../input/mines-vs-rocks/sonar.all-data.csv", header=None)
dataframe.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
92,0.0260,0.0192,0.0254,0.0061,0.0352,0.0701,0.1263,0.1080,0.1523,0.1630,...,0.0118,0.0120,0.0051,0.0070,0.0015,0.0035,0.0008,0.0044,0.0077,R
66,0.0265,0.0440,0.0137,0.0084,0.0305,0.0438,0.0341,0.0780,0.0844,0.0779,...,0.0038,0.0187,0.0156,0.0068,0.0097,0.0073,0.0081,0.0086,0.0095,R
39,0.0091,0.0213,0.0206,0.0505,0.0657,0.0795,0.0970,0.0872,0.0743,0.0837,...,0.0112,0.0112,0.0102,0.0026,0.0097,0.0098,0.0043,0.0071,0.0108,R
104,0.0307,0.0523,0.0653,0.0521,0.0611,0.0577,0.0665,0.0664,0.1460,0.2792,...,0.0321,0.0189,0.0137,0.0277,0.0152,0.0052,0.0121,0.0124,0.0055,M
150,0.0209,0.0278,0.0115,0.0445,0.0427,0.0766,0.1458,0.1430,0.1894,0.1853,...,0.0096,0.0014,0.0049,0.0039,0.0029,0.0078,0.0047,0.0021,0.0011,M


In [4]:
dataframe.shape
# So we have 208 samples
# a total of 60 featuers and 1 dependent variable

(208, 61)

#### Checking for missing values

In [5]:
# check for nan values ---> There are no null values
dataframe.isna().sum().sum()

0

#### Dependent Variable data distribution

In [6]:
dataframe[60].value_counts() # label is not skewed
# Mines -- 111 samples for mines
# Rock -- 97 samples for mines

M    111
R     97
Name: 60, dtype: int64

#### Segregating Dependent and Independent features

In [7]:
X = dataframe.drop(60, axis=1)
y = dataframe[60]

#### Target Varaible Encoding: Converting M,R to 1,0

In [8]:
y = pd.get_dummies(y, drop_first = True) # one column gets dropped (dummy variable trap)
y.sample(5) # R --> 1 and M --> 0

,R
157,0
167,0
61,1
178,0
144,0


In [9]:
# distributed of dependent categories
y.value_counts()

R
0    111
1     97
dtype: int64

#### Train-Test Split

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

In [11]:
X_train.shape, X_test.shape

((156, 60), (52, 60))

### Deep Learning (NN) without and with Dropout
1. Without Dropout
2. With Dropout

In [12]:
%%time
## Without Dropout
# Defining the architecture of model
model_without_dropout = keras.Sequential([
    keras.layers.Dense(60, input_dim = 60, activation='relu'), # first dense layer
    keras.layers.Dense(30, activation = 'relu'), # second dense layer
    keras.layers.Dense(15, activation = 'relu'), # 3rd dense layer
    keras.layers.Dense(1, activation = 'sigmoid') # output layer
])

model_without_dropout.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model_without_dropout.fit(X_train, y_train, epochs = 100, batch_size = 8)

2021-09-22 13:47:52.475611: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-22 13:47:52.478956: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-09-22 13:47:52.478997: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-22 13:47:52.479022: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (d532148e8218): /proc/driver/nvidia/version does not exist
2021-09-22 13:47:52.479411: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

Epoch 1/100
20/20 [==============================] - 1s 1ms/step - loss: 0.6926 - accuracy: 0.5265
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6692 - accuracy: 0.6853
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6293 - accuracy: 0.7857
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5765 - accuracy: 0.7943
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5433 - accuracy: 0.7910
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4295 - accuracy: 0.8903
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4473 - accuracy: 0.8460
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3838 - accuracy: 0.8526
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3666 - accuracy: 0.8885
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3165 - accuracy: 0.8706
Epoch 11/

In [13]:
# Here we can clearly see an accuracy of 1, which might be because of overfitting
model_without_dropout.evaluate(X_train, y_train)

5/5 [==============================] - 0s 2ms/step - loss: 0.0021 - accuracy: 1.0000


[0.0021449935156852007, 1.0]

In [14]:
# Evaluating without dropout model on test set
model_without_dropout.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.8944 - accuracy: 0.8269


[0.8943887948989868, 0.8269230723381042]

In [15]:
## prediction some samples
y_pred = model_without_dropout.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer i.e., 0 or 1 as the output Activation function was sigmoid (so output values are probabilities)
y_pred = np.round(y_pred)
print(y_pred[:10])

[2.18481176e-07 9.37286377e-01 2.29597092e-04 8.22868645e-02
 8.40270221e-01 8.93892407e-01 5.48772869e-06 9.82714772e-01
 9.99982119e-01 1.13333035e-05]
[0. 1. 0. 0. 1. 1. 0. 1. 1. 0.]


In [16]:
np.array(y_test[:10]).reshape(1,-1)

array([[0, 0, 1, 1, 0, 0, 0, 1, 1, 0]], dtype=uint8)

In [17]:
np.sum(np.array(y_test[:10]).reshape(1,-1) == y_pred[:10])
# So out of 10, 5 correctly classified

5

In [18]:
# printing the metrics
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.79      0.83        28
           1       0.78      0.88      0.82        24

    accuracy                           0.83        52
   macro avg       0.83      0.83      0.83        52
weighted avg       0.83      0.83      0.83        52



##### Model with dropout

In [19]:
%%time
## With Dropout
# Defining the architecture of model
model_with_dropout = keras.Sequential([
    keras.layers.Dense(60, input_dim = 60, activation='relu'), # first dense layer
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(30, activation = 'relu'), # second dense layer
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(15, activation = 'relu'), # 3rd dense layer
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(1, activation = 'sigmoid') # output layer
])

model_with_dropout.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model_with_dropout.fit(X_train, y_train, epochs = 100, batch_size = 8)

Epoch 1/100
20/20 [==============================] - 1s 1ms/step - loss: 0.6625 - accuracy: 0.5612
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6971 - accuracy: 0.5250
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7062 - accuracy: 0.5220
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6942 - accuracy: 0.5284
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6970 - accuracy: 0.5023
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7011 - accuracy: 0.4769
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6791 - accuracy: 0.5744
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6561 - accuracy: 0.5618
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6708 - accuracy: 0.5429
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6963 - accuracy: 0.5179
Epoch 11/

In [20]:
# evaluating the model on train data
model_with_dropout.evaluate(X_train, y_train)

# the accuracy is not 1, thus less chances of overfitting

5/5 [==============================] - 0s 1ms/step - loss: 0.1771 - accuracy: 0.9423


[0.17709912359714508, 0.942307710647583]

In [21]:
# evaluating the model on test data
model_with_dropout.evaluate(X_test, y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.5398 - accuracy: 0.7308


[0.5397859811782837, 0.7307692170143127]

In [22]:
# predicting some samples
y_pred = model_with_dropout.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

[0.00109255 0.84306324 0.06601304 0.56378424 0.88532424 0.8144927
 0.00980562 0.37254006 0.9837078  0.00484139]
[0. 1. 0. 1. 1. 1. 0. 0. 1. 0.]


In [23]:
np.sum(np.array(y_test[:10]).reshape(1,-1) == y_pred[:10])
# So out of 10, 5 correctly classified

5

In [24]:
# printing the metrics after dropout
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.71      0.74        28
           1       0.69      0.75      0.72        24

    accuracy                           0.73        52
   macro avg       0.73      0.73      0.73        52
weighted avg       0.73      0.73      0.73        52

